## Random Forest Classifier with Pipelines and Hyperparameter Tuning
Here we want to predict the 'time' feature for the 'tips' dataset

In [168]:
import seaborn as sns

In [169]:
df  = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [170]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

So this becomes a binary classification model

### Generic activities for the new data as automation
 1. Handling missing values 
 2. Handling categorical feature
 3. Handling outliers 
 4. Feature scaling 
 5. Automate all the above tasks

In [171]:
# Encoding the output feature 'time' using label encoder 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [172]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [173]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [174]:
# Independent and dependent features
x=df.drop(labels=['time'],axis=1)
y=df['time']

In [175]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [176]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [411]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [412]:
from sklearn.impute import SimpleImputer # Handling missing values 
from sklearn.preprocessing import StandardScaler # Feature scaling 
from sklearn.preprocessing import OneHotEncoder # Handling categorical feature
from sklearn.pipeline import Pipeline # Automation
from sklearn.compose import ColumnTransformer # Connecting to the pipeline

In [413]:
categorical_features = ['sex', 'smoker', 'day']
numerical_features = ['total_bill', 'tip', 'size']

### Feature Engineering Automation using Pipelines

In [414]:
numerical_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')), ## To handle the missing values
        ('scaler', StandardScaler()) ## For feature scaling 
    ]
)


categorical_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')), ## To handle the missing values
        ('onehotencoder', OneHotEncoder()) ## For converting categorical values to numerical values
    ]
)

In [415]:
preprocessor = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline, numerical_features),
    ('categorical_pipeline', categorical_pipeline, categorical_features)

])

In [416]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [417]:
x_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [418]:
x_test

array([[-0.04546101,  0.06468811, -0.61214068,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-1.30860871, -0.76316144, -0.61214068,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.4952348 , -0.76316144,  1.51942062,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.64841289,  1.45379161,  1.51942062,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.82506891, -0.76316144, -0.61214068,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.26608319, -0.76316144, -0.61214068,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.   

### Model Training

In [419]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [420]:
## Automate Model Training Process
models = {
        'Ramdom Forest' : RandomForestClassifier(),
        'Decision Tree' : DecisionTreeClassifier(),
        'SVC' : SVC()
}

In [421]:
from sklearn.metrics import accuracy_score

In [422]:
def evaluate_model(x_train, y_train, x_test, y_test, models):
    
    report = {}
    
    for i in range(len(models)):
        model = list(models.values())[i]
        
        ## Model Training
        model.fit(x_train, y_train) 
        
        ## Prediction
        y_pred = model.predict(x_test)
    
        ## Checking model accuracy 
        model_accuracy = accuracy_score(y_test, y_pred)
    
        #print('The accuracy score is - ')
        report[list(models.keys())[i]] = model_accuracy
    
    
    return report     

In [423]:
evaluate_model(x_train, y_train, x_test, y_test, models)

{'Ramdom Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877}

Choosing 'Random Forest' model for HyperParameter Tuning 

In [424]:
classifier = RandomForestClassifier()

In [426]:
parameters = {
              'max_depth' : [3,5,10,'none'],
              'n_estimators' : [100, 200, 300],
              'criterion' : ['gini', 'entropy']
             }

In [427]:
from sklearn.model_selection import RandomizedSearchCV

In [431]:
cv=RandomizedSearchCV(classifier,param_distributions=parameters,scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.974 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.949 total time=   0.6s
[CV 

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 341, in fit
    self._validate_params()
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 570, in _validate_params
    validate_parameter_constraints(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, 'none'],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [432]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 3, 'criterion': 'entropy'}

Training the model 'Random Forest' using the abive parameters

In [433]:
classifier = RandomForestClassifier(n_estimators = 300, max_depth = 3, criterion = 'entropy')

In [434]:
classifier.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=3, n_estimators=300)

In [435]:
y_pred = classifier.predict(x_test)

In [436]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])

In [438]:
## Checking model accuracy
accurancy = accuracy_score(y_test, y_pred)
print('The model accuracy is', accurancy)

The model accuracy is 1.0
